<a href="https://www.kaggle.com/code/prajyotbirajdar/horse-life-prediction?scriptVersionId=143887857" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv(r'../input/playground-series-s3e22/train.csv')
test_df = pd.read_csv(r'../input/playground-series-s3e22/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.columns

In [ ]:
train_df.drop('hospital_number',axis=1,inplace=True)
test_df.drop('hospital_number',axis=1,inplace=True)

In [ ]:

categorical_data_columns=['surgery','age','temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'surgical_lesion',
 'cp_data']



numerical_data_columns=[
  'rectal_temp','pulse','respiratory_rate',
  'nasogastric_reflux_ph','packed_cell_volume',
  'total_protein','abdomo_protein','lesion_1',
  'lesion_2','lesion_3']



In [ ]:
def fill_na(df):
#     categorical variable
    for column in categorical_data_columns:
        df[column].fillna(df[column].mode()[0],inplace=True)
# numericak variable
# don't use same name for iterator while doing iteration it will be fuck ur mind
    for col in numerical_data_columns:
        df[column].fillna(df[col].mean(),inplace=True)
    return df
   
train_df=fill_na(train_df)
test_df=fill_na(test_df)


In [ ]:
print(train_df["pain"].unique())
print(test_df['pain'].unique())

In [ ]:
test_df['pain']=test_df['pain'].replace('moderate','slight')

# Normalizing Numerical Columns

In [ ]:
from sklearn.preprocessing import StandardScaler
def normalise_num(df):
    sc = StandardScaler()
    sc.fit(df[numerical_data_columns])
    df[numerical_data_columns] = sc.transform(df[numerical_data_columns])
    return df

train_df = normalise_num(train_df)
test_df = normalise_num(test_df)



In [ ]:
# Categorical data manipulation
from sklearn.preprocessing import LabelEncoder
def categorical_data_conv(df):
    for col in categorical_data_columns:
        le=LabelEncoder()
        le.fit(df[col])
        df[col]=le.transform(df[col])
    return df
train_df = categorical_data_conv(train_df)
test_df = categorical_data_conv(test_df)
    

In [ ]:
train_df.head()

In [ ]:
# train test split dataset
x=train_df.drop(['id','outcome'],axis=1)
y=train_df['outcome']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=100)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
# Logistic regression
# for submission this model is giving best result
from sklearn.linear_model import LogisticRegression
model_l = LogisticRegression()
model_l.fit(x_train,y_train)



print("RSqured value on train:",model_l.score(x_train,y_train))
print("_"*50)
print("RSqured value on test:",model_l.score(x_test,y_test))


In [ ]:
test_data = test_df.drop(['id','outcome'],axis=1)


In [ ]:
test_data.head()

In [ ]:
output = model_l.predict(test_data)

In [ ]:
submission = pd.DataFrame(data={"id":test_df['id'],
                               'outcome': output})
submission.to_csv("horse_submission1.csv",index=False)
print("Done")
